In [0]:
dbutils.fs.put("/FileStore/tables/dynamicolumns.csv",
"""
id,name,loc,age,sex
1,ravi,bangalore,33,m
2,raj,chennai
3,mohan
4,prasad,hyderabad,35
5,sridhar,chennai
""",True)
     

Wrote 105 bytes.
Out[1]: True

In [0]:
df = spark.read.csv("/FileStore/tables/dynamicolumns.csv",header=True)
display(df)

id,name,loc,age,sex
1,ravi,bangalore,33,m
2,raj,chennai,null,null
3,mohan,null,null,null
4,prasad,hyderabad,35,null
5,sridhar,chennai,null,null


In [0]:
#same data without headers
dbutils.fs.put("/FileStore/tables/dynamicolumns_withoutheader.csv",
"""1,ravi,bangalore
2,raj,chennai,33,m
3,mohan
4,prasad,hyderabad,35,m,787878987
5,sridhar,chennai
""",True)

Wrote 96 bytes.
Out[3]: True

In [0]:
#same data without headers it will now read the first two columns since it have variable columns for rows
df1 = spark.read.csv("/FileStore/tables/dynamicolumns_withoutheader.csv")
display(df1)
     

_c0,_c1,_c2
1,ravi,bangalore
2,raj,chennai
3,mohan,null
4,prasad,hyderabad
5,sridhar,chennai


In [0]:

# Create Dataframe reading csv file using spark.read.text api
df1 = spark.read.text("/FileStore/tables/dynamicolumns_withoutheader.csv")
df1.show()

+--------------------+
|               value|
+--------------------+
|    1,ravi,bangalore|
|  2,raj,chennai,33,m|
|             3,mohan|
|4,prasad,hyderaba...|
|   5,sridhar,chennai|
+--------------------+



In [0]:

from pyspark.sql.functions import *
# Split text data using split function with comma delimieter
df3 =df1.select(split("value",",").alias("splitted_col"))
display(df3)

splitted_col
"List(1, ravi, bangalore)"
"List(2, raj, chennai, 33, m)"
"List(3, mohan)"
"List(4, prasad, hyderabad, 35, m, 787878987)"
"List(5, sridhar, chennai)"


In [0]:

# Get Length of each row using size function then find max length of row for generating no of columns dynamically
df3.select('splitted_col',size('splitted_col')).show(truncate=False)

+----------------------------------------+------------------+
|splitted_col                            |size(splitted_col)|
+----------------------------------------+------------------+
|[1, ravi, bangalore]                    |3                 |
|[2, raj, chennai, 33, m]                |5                 |
|[3, mohan]                              |2                 |
|[4, prasad, hyderabad, 35, m, 787878987]|6                 |
|[5, sridhar, chennai]                   |3                 |
+----------------------------------------+------------------+



In [0]:
for i in range (splitable_df.select(max(size("Splitable_col"))).collect([0][0])) :

In [0]:
#df3.select(max(size('splitted_col'))).first()[0]
# Verify no of columns is going to generate this from data.
df3.select(max(size('splitted_col'))).collect()[0][0]

Out[27]: 6

In [0]:
for i in range(df3.select(max(size('splitted_col'))).collect()[0][0]):
    # Dynamically Add Columns using WithColumn 
    df3=df3.withColumn('col'+str(i),df3["splitted_col"][i])
# Drop splitted_Col which is not required after splitting into individual columns
df3 = df3.drop("splitted_col")

   


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-31332495302255>:1
----> 1 for i in range(df3.select(max(size('splitted_col'))).collect()[0][0]):
      2     # Dynamically Add Columns using WithColumn 
      3     df3=df3.withColumn('col'+str(i),df3["splitted_col"][i])
      4 # Drop splitted_Col which is not required after splitting into individual columns

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3023, in DataFrame.select(self, *cols)
   2978 def select(self, *cols: "ColumnOrName") -> "Dat